# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [34]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [35]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [36]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt', encoding="utf8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt', encoding="utf8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [37]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [38]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    true_positives = len(gold.intersection(pred))
    predicted = len(pred)
    gold_standard = len(gold)
    
    precision = true_positives/predicted
    recall = true_positives/gold_standard
    F1 = (2*precision*recall)/(precision+recall)
    print(precision,recall,F1)
    pass

To test your code, you can run the following cell:

In [39]:
evaluation_report(set(range(3)), set(range(5)))

0.6 1.0 0.7499999999999999


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [40]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row.sentence_id,row.beg,row.end)

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [41]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [42]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

def span(df):
    sentence_id_unique = df.sentence_id.unique()
    for i in sentence_id_unique:
        sentence = df[df.sentence_id==i].sentence.values[0]
        doc = nlp(sentence)
        for ent in doc.ents:
            yield(i,ent.start,ent.end)

spans_dev_spacy = set(span(df_dev))

In [43]:
evaluation_report(spans_dev_gold,spans_dev_spacy)

0.529449922158796 0.6897076221057968 0.599045871559633


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [44]:
# TODO: Write code here to do your analysis
false_positives=spans_dev_spacy-spans_dev_gold
false_positives

{('1043-004', 30, 32),
 ('1014-010', 48, 49),
 ('1022-004', 9, 10),
 ('1079-005', 0, 3),
 ('0946-008', 22, 23),
 ('1094-014', 1, 2),
 ('1058-027', 31, 35),
 ('1101-012', 45, 49),
 ('1117-008', 8, 9),
 ('1066-022', 4, 5),
 ('1091-003', 20, 25),
 ('0948-039', 5, 7),
 ('1045-007', 27, 30),
 ('1160-014', 3, 4),
 ('0980-019', 46, 47),
 ('1044-005', 3, 5),
 ('1096-035', 20, 21),
 ('0960-004', 22, 23),
 ('1008-006', 29, 30),
 ('0973-000', 5, 6),
 ('1066-011', 5, 6),
 ('0960-014', 16, 17),
 ('1116-035', 0, 1),
 ('1017-002', 2, 3),
 ('1112-008', 0, 3),
 ('1046-005', 19, 20),
 ('1069-013', 14, 16),
 ('1000-010', 20, 21),
 ('1123-012', 5, 7),
 ('0957-019', 20, 25),
 ('1099-005', 7, 8),
 ('1051-011', 22, 24),
 ('0966-026', 6, 7),
 ('1060-010', 6, 7),
 ('1055-000', 0, 5),
 ('1155-009', 3, 4),
 ('1046-003', 15, 16),
 ('1157-002', 0, 6),
 ('0969-003', 20, 21),
 ('0966-158', 1, 2),
 ('1143-006', 18, 20),
 ('1058-041', 3, 4),
 ('1105-003', 4, 5),
 ('1038-007', 14, 17),
 ('1156-009', 2, 3),
 ('0955-005'

In [45]:
false_negatives=spans_dev_gold-spans_dev_spacy
false_negatives

{('1040-000', 0, 3),
 ('1065-031', 0, 1),
 ('1069-009', 5, 6),
 ('1058-037', 14, 15),
 ('0958-029', 2, 3),
 ('0994-010', 32, 34),
 ('1153-001', 0, 1),
 ('1067-012', 2, 5),
 ('1138-008', 13, 15),
 ('0949-005', 0, 1),
 ('1068-022', 2, 3),
 ('1089-000', 7, 10),
 ('1056-047', 3, 4),
 ('0999-003', 0, 1),
 ('1058-040', 29, 31),
 ('1068-010', 0, 2),
 ('1137-007', 6, 9),
 ('1101-012', 83, 85),
 ('0992-007', 1, 5),
 ('1107-000', 2, 3),
 ('1104-002', 26, 28),
 ('1087-000', 2, 3),
 ('1042-002', 24, 25),
 ('1022-004', 8, 9),
 ('1050-002', 1, 4),
 ('1135-006', 18, 20),
 ('1063-003', 2, 4),
 ('1016-007', 22, 23),
 ('0946-005', 14, 15),
 ('1078-005', 2, 3),
 ('0952-000', 7, 8),
 ('0972-007', 5, 6),
 ('1096-036', 23, 24),
 ('1101-012', 9, 11),
 ('1004-003', 7, 9),
 ('0957-015', 8, 10),
 ('1072-018', 100, 102),
 ('0986-006', 6, 7),
 ('1064-007', 21, 24),
 ('1106-000', 2, 3),
 ('0964-000', 2, 3),
 ('1030-018', 10, 11),
 ('0971-002', 13, 16),
 ('0958-029', 0, 1),
 ('1089-000', 2, 3),
 ('1157-001', 0, 1),

*TODO: Write a short text that discusses the errors that you observed*

Spacy's named entity recognizer interprets punctuations as tokens whereas in the gold standard punctuations are not considered to be tokens. Due to this the starting and ending position of named entities in gold standard class and predicted class does not match. In addition, Spacy's NER recognizes some extra entity types such as Ordinal,Cardinal,Money,Percent,Date and Time which is not considered in Gold standard class.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [46]:
# TODO: Write code here to improve the span prediction from Problem 2
def span2(df):
    sentence_id_unique = df.sentence_id.unique()
    for i in sentence_id_unique:
        sentence = df[df.sentence_id==i].sentence.values[0]
        doc = nlp(sentence)
        #docs = []
        #for token in doc:
        #    if token.is_punct is False:
        #        docs.append(token)
        #docs_joined = " ".join(str(i) for i in docs)
        #docs_nlp = nlp(docs_joined)
        for ent in doc.ents:
            if ent.label_ not in ['ORDINAL','CARDINAL','MONEY','PERCENT','DATE','TIME']:
                yield (i,ent.start,ent.end)
                
spans_dev_spacy2 = set(span2(df_dev))

In [47]:
evaluation_report(spans_dev_gold,spans_dev_spacy2)

0.8330267729409361 0.6888625992901808 0.7541165587419056


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [48]:
import numpy as np

df_dev_new=pd.DataFrame(list(spans_dev_spacy2))
df_dev_new.columns=['sentence_id','beg','end']
df_dev_new['label']='--NME--'

for index, row in df_dev_new.iterrows():
    df_dev_new.loc[index,'sentence']=df_dev[df_dev.sentence_id==row.sentence_id].sentence.values[0]
    
df_dev_new=df_dev_new[['sentence_id','sentence','beg','end','label']]
df_dev_new.head()

,sentence_id,sentence,beg,end,label
0,0962-010,"67 Mark Calcavecchia , Payne Stewart , Billy M...",4,6,--NME--
1,0949-001,LONDON 1996-08-30,0,1,--NME--
2,1043-004,Singapore has a mandatory death sentence for a...,30,32,--NME--
3,1140-003,""" I am sorry to my ancestors for five generati...",12,14,--NME--
4,1152-011,Clinton said Iraqi military forces overran the...,2,3,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [49]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row.sentence_id,row.beg,row.end,row.label)   

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [50]:
# TODO: Write code here to implement the baseline

def baseline(data):
    for i, row in data.iterrows():       
        row.label = "_".join(row.sentence.split(" ")[int(row.beg):int(row.end)])
        yield(row.sentence_id,row.beg,row.end,row.label)
        
evaluation_report(set(gold_mentions(df_dev)),set(baseline(df_dev_new)))    

0.3139178418148375 0.25959100895724185 0.2841813135985199


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [51]:
with bz2.open("kb.tsv.bz2", 'rt', encoding='utf8') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [52]:
df_kb.loc[df_kb["mention"] == "Sweden"]

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [60]:
# TODO: Write code here to implement the "most probable entity" method.
def most_probable_entity_1(mention):
    prob_entity = df_kb.loc[df_kb.mention == mention]
    if(len(prob_entity) > 0):
        return entity.iloc[0].entity
    else:
        return "--NME--"

def most_probable_entity(df):
    for i, row in df.iterrows():
        if " ".join(row.sentence.split(" ")[int(row.beg):int(row.end)]) in df_kb["mention"].values:
            row.label = df_kb.loc[df_kb["mention"] == " ".join(row.sentence.split(" ")[int(row.beg):int(row.end)]),'entity'].iloc[0]
            #print(df_kb.loc[df_kb["mention"] == " ".join(row.sentence.split(" ")[int(row.beg):int(row.end)]),'entity'].iloc[0])
        else:
            row.label = "--NME--"
            #print("--NME--")
        yield(row.sentence_id,row.beg,row.end,row.label)   

In [61]:
def gold_spans2(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row.sentence_id,row.beg,row.end,row.label)

In [62]:
evaluation_report(set(gold_spans2(df_dev)), set(most_probable_entity(df_dev_new)))

0.6409155937052933 0.5299983099543688 0.5802035152636448


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [56]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [57]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in 1970 and the second in ...
1,1970,FIFA_World_Cup,America 1975 and during the 1970 and 1978 Worl...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the 1990 World Cup
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the 1990 W...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the 1990 World Cup The...


From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [58]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
41470    the Humanities NEH based in Lincoln Nebraska H...
41471    town Hallam situated halfway between Lincoln a...
41472    games in Memorial Stadium in Lincoln The Huske...
41473    site and retail store in Lincoln Nebraska and ...
41474    The eastern segment begins in Lincoln and ends...
41475    1869 with one campus in Lincoln the system now...
41476    Ron Kurtenbach is Lincoln Nebraska communist b...
41477    state of Nebraska He represented Lincoln distr...
41478    national celebrity for his 1908 Lincoln to New...
41479    brewery founded in 1990 in Lincoln Nebraska US...
41480    state government agency Located in Lincoln the...
41481    Omaha Grand Island Kearney and Lincoln Nebrask.

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [68]:
# TODO: Write code here to implement the context-sensitive disambiguation method

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

dictionary = {}
for m in df_contexts.mention.unique():
    pipeline = Pipeline([('cv', CountVectorizer()),
                     ('mnb', MultinomialNB(class_prior = list(df_kb[df_kb.mention == m].sort_values(by=["entity"]).prob)))])
    pipeline.fit(df_contexts.context[df_contexts.mention == m], df_contexts.entity[df_contexts.mention == m])
    dictionary[m] = pipeline
    
def probable_entity(mention, sentence, beg, end, n):
    if mention in dictionary:
        splitSentence = sentence.split(" ")
        splitContext = splitSentence[max(0, int(beg) - n):min(len(splitSentence), int(end) + n)]
        context = ' '.join(splitContext)
        return dictionary[mention].predict([context])[0]
    return most_probable_entity_1(mention)
    
def mnb_prediction(df):
    for i in df.itertuples():
        splitLabel = i.sentence.split(" ")[int(i.beg):int(i.end)]
        label = " ".join(splitLabel)
        yield (i.sentence_id, i.beg, i.end, probable_entity(label, i.sentence, i.beg, i.end, 5))

evaluation_report(set(gold_mentions(df_dev)), set(mnb_predict(df_dev_new)))

0.6550173717555692 0.5416596248098698 0.5929694727104532


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>